In [15]:
import os 
import re
import pandas as pd 
import openai
import getpass
from llama_index import(VectorStoreIndex,
                        SimpleDirectoryReader,
                        load_index_from_storage,
                        StorageContext,
                        ServiceContext,
                        Document)
from llama_index.text_splitter import SentenceSplitter
from llama_index.extractors import TitleExtractor 
from llama_index.embeddings import OpenAIEmbedding
from llama_index.schema import TransformComponent
from llama_index.ingestion import IngestionPipeline

In [5]:
default_path = os.getcwd()
data_path = os.path.join('/workspace/data/')

In [16]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key: ········


In [49]:
documents[1].text

'2 \n 차    례 \n \n제1장 총 칙 \n제1조(목적) ·······························  5 \n제2조(적용범위 ) ·····························  5 \n제3조(사원의 정의) ···························  5 \n \n제2장 채용 및 근로계약  \n제4조(채용기회 ) ·····························  5 \n제5조(전형 및 채용서류 ) ·························  5 \n제6조(근로계약 ) ·····························  5 \n제7조(수습기간 ) ·····························  5 \n \n제3장 복 무 \n제8조(복무의무 ) ·····························  6 \n제9조(출근, 결근) ····························  6 \n제10조(지각․조퇴 및 외출) ························  6 \n제11조(공민권행사  및 공의 직무 수행) ··················  6 \n제12조(출장) ······························  6 \n \n제4장 인사 \n제1절 인사위원회  \n제13조(인사위원회의  구성) ························  7 \n제14조(위원회의  기능) ··························  7 \n제15조(위원회의  소집 및 운영) ······················  7 \n \n     제2절 배치・전직 및 승진 \n제16조(배치, 전직, 승진) ························  7 \n \n제3절 휴직 및 복직 \n제17조(휴직) ······························  7 \n제18조(육아휴직 ) ····························  8 \n제19조(가족돌봄휴직  등)  ······················

In [50]:
class TextCleaner(TransformComponent):
    def __call__(self, nodes, **kwargs):
        try:
            assert nodes is not NULL
        except:
            return nodes
            
        for node in nodes:
            try:
                node.text = re.sub(r'[^0-9a-zA-Zㄱ-ㅣ가-힣 ]', '', node.text)
            except:
                print('err')
                continue 

In [51]:
node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=30)
extractor = TitleExtractor()

In [52]:
pipeline = IngestionPipeline(
    transformations=[
        node_parser,
        TextCleaner(), 
        # embed_model,
    ],
)

In [53]:
nodes = pipeline.run(documents=documents[:4])
nodes[0]

TextNode(id_='d31da2e6-9855-4c3c-8223-6128f59ef128', embedding=None, metadata={'page_label': '1', 'file_name': '취업규칙.pdf', 'file_path': '/workspace/data/pdf/rules/취업규칙.pdf', 'file_type': 'application/pdf', 'file_size': 520849, 'creation_date': '2024-01-19', 'last_modified_date': '2024-01-19', 'last_accessed_date': '2024-01-19'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='105acaff-d507-47fb-8329-8a8a1f9de4af', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': '취업규칙.pdf', 'file_path': '/workspace/data/pdf/rules/취업규칙.pdf', 'file_type': 'application/pdf', 'file_size': 520849, 'creation_date': '2024-01-19', 'last_modified_date': '2024-01-19', 'last_accessed_date': '2024-01-19'}

In [60]:
txt = re.sub(r'[^0-9a-zA-Zㄱ-ㅣ가-힣 ()]', '', documents[1].text)
txt = re.sub(r' +', ' ', txt)
txt

'2 차 례 제1장 총 칙 제1조(목적) 5 제2조(적용범위 ) 5 제3조(사원의 정의) 5 제2장 채용 및 근로계약 제4조(채용기회 ) 5 제5조(전형 및 채용서류 ) 5 제6조(근로계약 ) 5 제7조(수습기간 ) 5 제3장 복 무 제8조(복무의무 ) 6 제9조(출근 결근) 6 제10조(지각조퇴 및 외출) 6 제11조(공민권행사 및 공의 직무 수행) 6 제12조(출장) 6 제4장 인사 제1절 인사위원회 제13조(인사위원회의 구성) 7 제14조(위원회의 기능) 7 제15조(위원회의 소집 및 운영) 7 제2절 배치전직 및 승진 제16조(배치 전직 승진) 7 제3절 휴직 및 복직 제17조(휴직) 7 제18조(육아휴직 ) 8 제19조(가족돌봄휴직 등) 8 제20조(가족돌봄 등을 위한 근로시간 단축) 8 제21조(복직) 8 제22조(근속기간의 계산) 9 제5장 근로조건 제1절 근로시간 제23조(근무형태 ) 9 제24조(근로시간 ) 9 제25조(휴게) 9 제26조(간주근로시간제 ) 9 제27조(연장야간 및 휴일근로 ) 9 제28조(야간 및 휴일근로의 제한) 10 제29조(근로시간 및 휴게 휴일의 적용제외 ) 10 '

In [44]:
re.sub('\n', '', documents[13].text)
# re.sub(r'[^0-9a-zA-Zㄱ-ㅣ가-힣 ]', '', node.text)

'14  30분씩 유급 수유시간을  준다.   제 6 장 임 금  제45조 (임금의 구성항목 )  사원에 대한 임금은 연간 일정시간  분의 시간외 근로수당  등을 포함하는  포괄임금제를 적용한다 .  제46조 (임금의 계산 및 지급방법 )  ① 임금 중 기본연봉은  금액을 12개월로 나눠 매월 지급하는  방식으로  하되, 결근일에  대하여는  해당일분 과 유급 주휴일분 을 감하여 지급한다 .  명절상여금은  설과 추석에 연 2회 나누어 지급한다 . ① 임금은 매월 초일부터  말일까지를  산정기간으로  하여 해당 월의 말일에 사원이 지정한 사원 명의의 예금계좌에  입금하여  지급한다 . 다만, 지급일이  토요일 또는 공휴일인  경우에는  그 전일에 지급한다 . ② 신규채용 , 승진, 전보, 퇴직 등의 사유로 임금을 정산하는  경우에는  발령일을  기준으로  그 월액을 일할 계산하여  지급한다 .  제47조 (비상시 지급)  사원이 다음 각 호의 사유로 청구하는  경우에는  지급기일  전이라도  이미 제공한 근로에 대한 임금을 지급한다 .  1. 사원 또는 그의 수입에 의하여 생활을 유지하는  자의 출산, 질병 또는 재해의 비용에 충당하는  경우 2. 사원 또는 그의 수입에 의하여 생활하는  자의 혼인 또는 사망 시 그 비용에 충당하는  경우 3. 사원이 부득이한  사정으로  1주일 이상 귀향하는  경우  제48조 (휴업수당 )  ① 회사의 귀책사유로  휴업하는  경우에는  휴업기간  동안 사원에게  평균임금의  100분 70의 수당을 지급한다 . 다만, 평균임금의  100분의 70에 해당하는  금액이 통상임금을  초과하는  경우에는  통상임금으로  지급한다 . ② 부득이한  사유로 사업을 계속하는  것이 불가능한  경우에는  노동위원회의  승인을 받아 제1항에 정한 금액에 못 미치는 휴업수당을  지급 할 수 있다.  제49조 (상여금지급 )  회사는 별도의 규정 및 기준에 의해 상여금을  지급할 수 있다.   제 7 장 퇴직․해고 등  제50조 (퇴직 및 퇴직일

In [28]:
documents[0].text

'1 \n 문서번호  FG–1601 \n제정일자 - \n개정일자  2022.09.01 \n \n \n \n \n \n \n \n \n \n \n취 업 규 칙 \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n주식회사  핑거 \n \n \n \n \n \n \n \n \n \n '